## ResNet50

In [ ]:
from keras.preprocessing.image import ImageDataGenerator


from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import ResNet50

import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import datetime

from sklearn.metrics import accuracy_score

In [ ]:
trainImagePath = '../dataSet/splitImages/train'
validationImagePath = '../dataSet/splitImages/val'
testImagePath = '../dataSet/splitImages/test'
checkpointPath = '../checkpoints/epoch_{epoch:04d}.ckpt'
logsPath = '../logs/fit/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
bestCheckpointPath = '../model/best/best_model'

batchSize = 64
imageWidth = 224
imageHeight = 224
imageChannel = 3


### 이미지 데이터 생성

ImageDataGenerator (train, validation)

In [ ]:
trainDataGen = ImageDataGenerator(rescale=1./255,
                                  rotation_range = 30,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=20,
                                  zoom_range=0.3,
                                  horizontal_flip=False,
                                  vertical_flip=False,
                                  brightness_range=[0.7, 1.3],
                                  fill_mode='nearest',
                                  )

train set

In [ ]:
trainGenSet = trainDataGen.flow_from_directory(
    trainImagePath,
    batch_size=batchSize,
    target_size=(imageWidth,imageHeight),
    class_mode='categorical',
    shuffle=True
)

validation set

In [ ]:
validationGenSet = trainDataGen.flow_from_directory(
    validationImagePath,
    batch_size=batchSize,
    target_size=(imageWidth,imageHeight),
    class_mode='categorical',
)

ImageDataGenerator (train, validation)

In [ ]:
testDataGen = ImageDataGenerator(rescale=1./255)

test set

In [ ]:
testGenSet = testDataGen.flow_from_directory(
    testImagePath,
    batch_size=batchSize,
    target_size=(imageWidth,imageHeight),
    class_mode='categorical'
)

make model

In [ ]:
model = tf.keras.Sequential([
  ResNet50(include_top=False, weights='imagenet', input_shape=(imageWidth, imageHeight, imageChannel)),
  GlobalAveragePooling2D(),
  Dropout(rate=0.0),
  Dense(1049, activation='relu' ),
  Dense(44, activation='softmax'),
])

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

difine callbacks

In [ ]:
checkpointPath = '../checkpoints/epoch_{epoch:04d}.ckpt'

In [ ]:
checkpoint = ModelCheckpoint(
                        checkpointPath, monitor='val_loss', verbose=0, save_best_only=False,
                        save_weights_only=False, mode='auto', save_freq='epoch'
                    )


In [ ]:
earlyStopping = [
                    EarlyStopping(monitor='val_loss', patience=10, ),
                    ModelCheckpoint( bestCheckpointPath, monitor='val_loss', save_best_only=True )
                ]

In [ ]:
tensorboard = tf.keras.callbacks.TensorBoard(logsPath, histogram_freq=1)

In [ ]:
epochs = 200
history = model.fit(
    trainGenSet,
    epochs=epochs,
    validation_data=validationGenSet,
    callbacks=[checkpoint,earlyStopping, tensorboard],
    verbose=1
)

In [ ]:
scores = model.evaluate(testGenSet)
print(scores)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label = 'train loss')
loss_ax.plot(history.history['val_loss'], 'r', label = 'val loss')
acc_ax.plot(history.history['accuracy'], 'b', label = 'train accuracy')
acc_ax.plot(history.history['val_accuracy'], 'g', label = 'val accuracy')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_xlabel('accuracy')

loss_ax.legend(loc = 'upper left')
acc_ax.legend(loc = 'lower left')

plt.show()

In [ ]:
model.save('../model/resnetModel')